In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [2]:
gmsh.open("body2.geo")

Info    : Reading 'body2.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000629244s, CPU 0.000631s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000251884s, CPU 0.000251s)
Info    : Meshing 3D

In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat])

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 386)


LowLevelFEM.Problem("body2", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321)

In [4]:
supp = FEM.displacementConstraint("left", ux=0, uy=0, uz=0)
load = FEM.load("right", fx=1)

("right", 1, 0, 0)

In [5]:
K = FEM.stiffnessMatrix(problem)
f = FEM.loadVector(problem, [load])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :f3D)

In [6]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])

In [7]:
q1 = FEM.solveDisplacement(K, f)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; -5.625000002570498e-7; -5.62499999941991e-7;;], [0.0], Int64[], 1, :u3D)

In [8]:
FEM.showDoFResults(problem, q1, :uvec, visible=true)

1

In [9]:
q2 = FEM.solveDisplacement(problem, [load], [supp])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; -5.625000002570498e-7; -5.62499999941991e-7;;], [0.0], Int64[], 1, :u3D)

In [10]:
FEM.showDoFResults(problem, q2, :uvec)

2

In [11]:
elsupp = FEM.elasticSupport("left", kx=1e6, ky=1e6, kz=1e6)

("left", 1.0e6, 1.0e6, 1.0e6)

In [12]:
q4 = FEM.solveDisplacement(problem, [load], [], [elsupp])

LowLevelFEM.VectorField(Matrix{Float64}[], [1.299265857591369e-6; 2.564102694320846e-7; … ; -5.625000001994694e-7; -5.624999998885229e-7;;], [0.0], Int64[], 1, :u3D)

In [13]:
FEM.showDoFResults(problem, q4, :uvec)

3

In [14]:
q3 = copy(q2)
q3.a[1] = 5
q2.a[1] = 3
q3.a[1]

5.0

In [15]:
q2 + q3
q2 - q3
q2 * 2
2 * q2
q2 / 2

LowLevelFEM.VectorField(Matrix{Float64}[], [1.5; 0.0; … ; -2.812500001285249e-7; -2.812499999709955e-7;;], [0.0], Int64[], 1, :u3D)

In [16]:
A1 = FEM.solveStrain(problem, q1)

LowLevelFEM.TensorField([[3.7446247155861753e-6; 1.3851318264023598e-6; … ; 6.18405728896586e-8; -1.4299106321137822e-6;;], [5.133026643519805e-6; -4.5918775942833095e-7; … ; 1.0096309599970558e-8; -1.4776574859702471e-6;;], [5.025810444459495e-6; -6.841721803063807e-8; … ; 7.459255750614673e-11; -1.4982402288169446e-6;;], [5.00203429265627e-6; -3.4073685252966774e-9; … ; -1.0044044806630302e-10; -1.4999719035390086e-6;;], [5.000123437131989e-6; 1.5576390170739414e-10; … ; -9.223272747254567e-12; -1.5000103746277933e-6;;], [5.0000150295273184e-6; 2.0926500640828784e-11; … ; 1.8488057889888729e-13; -1.5000013391438653e-6;;], [5.000003209319867e-6; -3.5892013886436585e-12; … ; 1.0595661441734569e-13; -1.500000074292642e-6;;], [5.000000591887934e-6; -1.2084819771404284e-12; … ; 8.371152502331365e-15; -1.5000000007641868e-6;;], [5.0000000894281525e-6; -1.9056532424250672e-13; … ; -1.486322593865011e-16; -1.4999999998725372e-6;;], [5.0000000120014264e-6; -2.3626352678202296e-14; … ; -9.5973

In [17]:
A2 = FEM.solveStrain(problem, q1, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [6.204514322733413e-6; 3.49101654052599e-6; … ; 5.505714157152952e-21; -1.5000000000000257e-6;;], [0.0], Int64[], 1, :e)

In [18]:
FEM.showStrainResults(problem, A1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


4

In [19]:
FEM.showDoFResults(problem, A2, :e)

5

In [20]:
A3 = FEM.elementsToNodes(problem, A1)

LowLevelFEM.TensorField(Matrix{Float64}[], [6.204514322733413e-6; 3.49101654052599e-6; … ; 5.505714157152952e-21; -1.5000000000000257e-6;;], [0.0], Int64[], 1, :e)

In [21]:
FEM.showDoFResults(problem, A3, :e)

6

In [22]:
S1 = FEM.solveStress(problem, q1)

LowLevelFEM.TensorField([[1.0081681926578165; 0.10654860203095076; … ; 0.004756967145358354; 0.0023888308595453045;;], [0.9940006235281256; -0.03532213534064084; … ; 0.0007766391999977353; -0.005564414934470435;;], [0.9985925495589456; -0.005262862925433698; … ; 5.737889038934364e-6; -0.0007170019848763143;;], [0.9995717950658349; -0.0002621052711766675; … ; -7.72618831279254e-6; -1.1571908538898779e-5;;], [0.999926922409242; 1.1981838592876473e-5; … ; -7.094825190195821e-7; 5.791438610680544e-6;;], [0.9999914202398474; 1.609730818525291e-6; … ; 1.42215829922221e-8; 6.213018436951323e-7;;], [0.9999991610817622; -2.7609241451105066e-7; … ; 8.150508801334284e-9; 5.573410344350549e-10;;], [0.999999916140597; -9.296015208772526e-8; … ; 6.439348078716435e-10; -6.238942334979569e-9;;], [0.999999989917305; -1.465887109557744e-8; … ; -1.1433250722038546e-11; -6.736745328250748e-10;;], [0.999999998586056; -1.8174117444770997e-9; … ; -7.382543699126807e-12; -1.1883299899650979e-11;;]  …  [1.0000

In [23]:
S2 = FEM.solveStress(problem, q1, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [1.6704461638128423; 0.2685397338866146; … ; 4.2351647362715017e-16; -4.163336342344337e-16;;], [0.0], Int64[], 1, :s)

In [24]:
FEM.showStressResults(problem, S1, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


7

In [25]:
FEM.showDoFResults(problem, S2, :s)

8

In [26]:
S3 = FEM.elementsToNodes(problem, S1)

LowLevelFEM.TensorField(Matrix{Float64}[], [1.6704461638128423; 0.2685397338866146; … ; 4.2351647362715017e-16; -4.163336342344337e-16;;], [0.0], Int64[], 1, :s)

In [27]:
FEM.showDoFResults(problem, S3, :s)

9

In [28]:
thermo = FEM.Problem([mat], type=:HeatConduction)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 386)
Info    : Renumbering nodal model data (9 steps)


LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321)

In [29]:
temp = FEM.temperatureConstraint("left", T=0)
flux = FEM.heatFlux("right", qn=1)

flx = FEM.heatFluxVector(thermo, [flux])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :qn)

In [30]:
KT = FEM.heatConductionMatrix(thermo)

FEM.applyBoundaryConditions!(thermo, KT, flx, [temp])

T1 = FEM.solveTemperature(KT, flx)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; -0.20555555555552796; -0.2166666666666383;;], [0.0], Int64[], 1, :T)

In [31]:
FEM.showDoFResults(thermo, T1, :T)

10

In [32]:
T2 = FEM.solveTemperature(thermo, [flux], [temp])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; -0.20555555555552796; -0.2166666666666383;;], [0.0], Int64[], 1, :T)

In [33]:
FEM.showDoFResults(thermo, T2, :T)

11

In [34]:
qT1 = FEM.solveHeatFlux(thermo, T2)

LowLevelFEM.VectorField([[0.9999999999998873; 0.0; … ; 4.683753385137379e-16; 1.5612511283791264e-16;;], [0.9999999999998853; 3.747002708109903e-15; … ; -6.245004513516506e-16; 0.0;;], [0.9999999999998815; 7.494005416219807e-15; … ; 2.4980018054066022e-15; 1.5404344466680216e-15;;], [0.9999999999998715; 4.9960036108132044e-15; … ; 3.608224830032833e-16; -2.4980018054066022e-15;;], [0.9999999999998865; -4.9960036108132044e-15; … ; 2.4980018054066022e-15; -2.4980018054066022e-15;;], [0.9999999999998616; 4.9960036108132044e-15; … ; -2.4980018054066022e-15; -3.830269434954238e-15;;], [0.9999999999998765; 0.0; … ; 3.2196467714132044e-15; 0.0;;], [0.9999999999998566; 6.661338147776226e-15; … ; -7.216449660066265e-16; 8.271161533459161e-15;;], [0.9999999999998466; 0.0; … ; -4.9960036108132044e-15; -4.9960036108132044e-15;;], [0.9999999999998715; -4.9960036108132044e-14; … ; 1.9984014443252818e-14; 2.4980018054066022e-15;;]  …  [0.9999999999998765; -2.9976021664879227e-14; … ; 7.49400541621980

In [35]:
FEM.showElementResults(thermo, qT1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


12

In [36]:
qT2 = FEM.solveHeatFlux(thermo, T2, DoFResults=true)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.9999999999998881; 0.0; … ; 1.4988010832439613e-14; -1.4988010832439613e-14;;], [0.0], Int64[], 1, :q3D)

In [37]:
FEM.showDoFResults(thermo, qT2, :qvec)

13

In [38]:
qT3 = FEM.elementsToNodes(thermo, qT1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.9999999999998881; 0.0; … ; 1.4988010832439613e-14; -1.4988010832439613e-14;;], [0.0], Int64[], 1, :q3D)

In [39]:
FEM.showDoFResults(thermo, qT3, :qvec)

14

In [40]:
heatconvB = FEM.heatConvection("bottom", h=1, Tₐ=0)
heatconvT = FEM.heatConvection("top", h=100, Tₐ=20)
heatconvF = FEM.heatConvection("front", h=1, Tₐ=0)
heatconvR = FEM.heatConvection("rear", h=1, Tₐ=0)

("rear", 1, 0, 0 + 2im)

In [41]:
T3 = FEM.solveTemperature(thermo, [], [temp], [heatconvB, heatconvT, heatconvF, heatconvR])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 19.265254842451537; 19.265352202702054;;], [0.0], Int64[], 1, :T)

In [42]:
FEM.showDoFResults(thermo, T3, :T)

15

In [43]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [44]:
gmsh.finalize()